In [1]:
from openai import OpenAI
import json

client = OpenAI(api_key="sk-988eb73686e24998a62dd62cdfdc19ac", base_url="https://api.deepseek.com")

In [2]:
def split_file(file_path, n):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    length = len(content)
    chunk_size = length // n
    chunks = [content[i:i + chunk_size] for i in range(0, length, chunk_size)]
    if len(chunks) > n:
        chunks[-2] += chunks[-1]
        chunks.pop()
    return chunks

def get_gpt_responses(chunks, sample, prompt0):
    responses = []
    i = 0
    for chunk in chunks:
        prompt = chunk + prompt0 
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": sample["system"]},
                {"role": "user", "content": sample["user"]},
                {"role": "assistant", "content": sample["assistant"]},
                {"role": "user", "content": prompt}
            ],
            stream=False,
            temperature=1.0
        )
        responses.append(response.choices[0].message.content.strip())
        i += 1
        print(f"Chunk {i} of {len(chunks)} processed.")
    return responses

def main(file_path, n, sample_path, prompt0):
    with open(sample_path, 'r', encoding='utf-8') as sample_file:
        sample = json.load(sample_file)
    
    chunks = split_file(file_path, n)
    responses = get_gpt_responses(chunks, sample, prompt0)
    return responses

def main2(file_path, n, prompt0):
    chunks = split_file(file_path, n)
    responses = []
    i = 0
    for chunk in chunks:
        prompt = chunk + prompt0 
        response = client.chat.completions.create(
            model="deepseek-chat",
            messages=[
                {"role": "system", "content": "Hello! I'm an AI assistant. How can I help you today?"},
                {"role": "user", "content": "I need help with something."},
                {"role": "assistant", "content": "Sure! I'll do my best to help you."},
                {"role": "user", "content": prompt}
            ],
            stream=False,
            temperature=1.0
        )
        responses.append(response.choices[0].message.content.strip())
        i += 1
        print(f"Chunk {i} of {len(chunks)} processed.")
    return responses

#### 生成词汇
耗时1min

In [ ]:
# 调用主函数
prompt1 = "\n\n Find a new word in the text above and return its English definition in the format 'xxx : xxx' ,Please choose nouns, adjectives, adverbs, or verbs"
prompt2 = "\n\n Find a word in the text above which is rare but esay to guess by the text, and return your brief guess in the format 'xxx : xxx' ,Please choose nouns, adjectives, adverbs, or verbs"
a = main('a.txt', 14, 'sample1.json' , prompt1)
for i in a:
    print(i)
print("---------------") 
a = main('a.txt', 20, 'sample2.json' , prompt2) 
for i in a:
    print(i)

#### questions in RR
这个效果不好,别用

In [ ]:
prompt3 = "If you don't fully understand the story, please ask 5 Yes/No questions with the about the story. It is best if each of questions consists of multiple logically related questions. no longer than 70 words in a.Please express it in a question"
a = main('a.txt', 1, 'sample3.json' , prompt3)

#### ideas in RR
这个效果不好,别用

In [ ]:
# 这个效果不好,别用
prompt4 = "\n\n Give me 3 ideas from this reading that would be interesting to discuss,no longer than 70 words "
a = main2('a.txt', 1, prompt4)

#### 一键生成小说大意(人物介绍+切片分段大意(中英文))

In [7]:
# 设置文件名
title_of_your_md = "Johnny Tremain 1-2"
md_file_name = f"{title_of_your_md}.md"

# 准备提示
prompt5 = "\n\n  用中文帮我理一下这本小说的人物和人际关系"
prompt6 = "\n\n  用中文给这一段做一个故事梗概,多补充一点细节分点列出,并在你认为重点的地方给出英文对照"

with open(md_file_name, 'w', encoding='utf-8') as md_file:
    md_file.write(f"# {title_of_your_md}\n\n")
    md_file.write("## 人物和人际关系\n\n")
    a = main2('a.txt', 1, prompt5)
    for i in a:
        md_file.write(i + "\n")
    md_file.write("<br/>\n\n")
    md_file.write("## 故事梗概\n\n")
    a = main('a.txt', 12, 'sample_for_preparing_quiz.json', prompt6)
    j = 0
    for i in a:
        j += 1
        md_file.write(f"### 分段{j}\n")
        md_file.write(i + "\n")
        md_file.write("<br/>\n")

print(f"输出已保存到 {md_file_name}")

Chunk 1 of 1 processed.
Chunk 1 of 12 processed.
Chunk 2 of 12 processed.
Chunk 3 of 12 processed.
Chunk 4 of 12 processed.
Chunk 5 of 12 processed.
Chunk 6 of 12 processed.
Chunk 7 of 12 processed.
Chunk 8 of 12 processed.
Chunk 9 of 12 processed.
Chunk 10 of 12 processed.
Chunk 11 of 12 processed.
Chunk 12 of 12 processed.
输出已保存到 Johnny Tremain 1-2.md


#### text-sample转化器(开发用)

In [ ]:
multiline_text = """ 这一段描述了波士顿清晨的景象，以及约翰尼·特雷曼（Johnny Tremain）和他的两个学徒伙伴多夫（Dove）和达斯蒂·米勒（Dusty Miller）在拉帕姆（Lapham）银匠店的一天开始。以下是故事梗概的详细分点：

1. **清晨的波士顿**：
   - 海鸥在岩石岛屿上醒来，飞到城镇上寻找食物，开始尖叫和争吵。
- Seagulls wake up on the rocky islands, fly to the town in search of food, and begin to scream and quarrel.
   - 波士顿的公鸡早已唤醒了黎明，母鸡也开始活动。
- Boston's roosters have long awakened the dawn, and the hens have begun to move.
   - 猫在各种场所捕捉最后一只老鼠，准备睡觉。
- Cats catch the last mouse in various places and prepare for bed.
   - 马在马厩里摇动缰绳，发出嘶鸣。
- Horses shake the reins and neigh in the stable.
   - 牛在谷仓里低声叫唤，等待挤奶。
- Cows call in the barn, waiting to be milked.
   - 波士顿慢慢醒来，阳光从东方水平照射进来，教堂的钟声响起，提醒人们起床开始新的一天。
- Boston slowly wakes up, the sun shines horizontally from the east, and the church bells ring, reminding people to get up and start a new day.

2. **拉帕姆家的早晨**：
   - 在汉考克码头拥挤的鱼街的一栋小房子里，拉帕姆夫人站在梯子下，呼唤睡在阁楼上的**学徒们**起床。
- In a small house on the crowded Fish Street at Hancock Wharf, Mrs. Lapham stood at the bottom of the ladder and called the **apprentices** sleeping in the attic to get up.
   - 学徒们包括**约翰尼·特雷曼、多夫和达斯蒂·米勒**。约翰尼是十四岁的瘦弱男孩，但他在学徒中有着领导地位。
- The apprentices included **Johnny Tremaine, Dov and Dusty Miller**. Johnny was a thin boy of fourteen, but he had a leadership position among the apprentices.
   - 拉帕姆夫人命令约翰尼帮助叫醒其他两个学徒，并让他们去取水准备早餐。
- Mrs. Lapham ordered Johnny to help wake up the other two apprentices and asked them to fetch water and prepare breakfast.

3. **学徒们的互动**：
   - 约翰尼迅速起床，穿好衣服，站在阁楼的窗户前欣赏汉考克码头的景色。
- Johnny quickly got up, got dressed, and stood in front of the attic window to enjoy the view of Hancock Wharf.
   - 多夫和达斯蒂慢慢起床，多夫对约翰尼的领导地位感到不满，但最终还是听从了命令。
- Dov and Dusty got up slowly, and Dov was dissatisfied with Johnny's leadership, but finally obeyed the order.
   - 约翰尼和多夫之间发生了争执，约翰尼威胁要再次打败多夫，因为他不小心**绊倒**了他。
- There was an argument between Johnny and Dov, and Johnny threatened to beat Dov again because he accidentally **tripped** him.
   - 达斯蒂对两个年长的学徒的争吵感到害怕，他崇拜约翰尼，但对多夫也有同情。
- Dusty is frightened by the quarrel between the two older apprentices. He idolizes Johnny but also sympathizes with Dove.

4. **学徒们的日常**：
   - 约翰尼在学徒中有着特殊的地位，因为他聪明、可靠，且在银匠工作上非常熟练。
- Johnny has a special status among the apprentices because he is smart, reliable, and very skilled in silversmithing.
   - 多夫对约翰尼的领导地位感到**嫉妒**，经常计划要报复，但从未真正实施。
- Dove is **jealous** of Johnny's leadership and often plans revenge, but never really carries it out.
   - 达斯蒂是十一岁的学徒，他对约翰尼和多夫的争吵感到困惑，但仍然忠于约翰尼。
- Dusty is an eleven-year-old apprentice who is confused by the quarrel between Johnny and Dove but remains loyal to Johnny.
这一段通过描述波士顿的清晨和学徒们的日常生活，展现了约翰尼·特雷曼的**领导能力**和他与同伴之间的关系。
This section shows Johnny Tremain's **leadership** and his relationship with his companions by describing the early morning in Boston and the daily life of the apprentices.


将以上这一段中英文笔记一字不落地写进代码块写成一行,然后返回给我,换行符用\n转义 """

# 将多行文字转换为一行，换行符用\n表示
single_line_text = multiline_text.replace('\n', '\\n')

# 将结果写入到文件 b.txt
with open('b.txt', 'w', encoding='utf-8') as file:
    file.write(single_line_text)

print("文本已经写入到 b.txt 文件中。")